# Aries Basic Controller Example - Bob
## DID Exchange - Invitee

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the Alice notebook ([localhost:8888]("http://localhost:8888"))</b>


6. Pull in dependencies
7. Instatiate the controller for our Agent
8. Set up a message listener running as a service on our controller
9. Paste the invitation from the Alice notebook into the invitation variable here
10. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Send Trust Ping to the Alice


### 6. Pull in dependencies


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 7. Instatiate the controller for our Agent

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 8. Set up a listener for basicmessages events emitted by the controller as it receives webhooks

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle messages ", payload, connection_id)

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

### 9. Paste the invitation from the Alice notebook into the invitation variable here;

In [4]:
#Paste in invitation from Alice agent
invite = {'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '67124ffd-43f4-4161-9926-ec815f1376e3', 'recipientKeys': ['9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr'], 'serviceEndpoint': 'http://172.17.0.1:8020', 'label': 'Alice'}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNjcxMjRmZmQtNDNmNC00MTYxLTk5MjYtZWM4MTVmMTM3NmUzIiwgInJlY2lwaWVudEtleXMiOiBbIjl5d3hxUEt0YXlYcHlpUTdLOFllZEx3SnE1b3NHcEo2SFVFTHZkQ3Y0aGRyIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE3Mi4xNy4wLjE6ODAyMCIsICJsYWJlbCI6ICJBbGljZSJ9'}

### 10. Accept the invitation, then move to Alice's notebook

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection {'created_at': '2020-07-03 11:44:14.565124Z', 'invitation_key': '9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr', 'request_id': 'a7a7e8a8-1981-4a69-ab45-d6ea82a6dc04', 'their_label': 'Alice', 'updated_at': '2020-07-03 11:44:14.581481Z', 'accept': 'manual', 'my_did': 'WMt1wVYjkRNGsWt2jaXvLS', 'invitation_mode': 'once', 'initiator': 'external', 'connection_id': '235a8f72-feef-44b9-ad15-d417e1ceeff9', 'routing_state': 'none', 'state': 'request'}


### 14. Send Trust Ping to the Alice

In [6]:
# Do a trust ping to ensure connection is accepted by alice
trust_ping = await agent_controller.messaging.trust_ping(alice_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - Bob -> Alice")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - Bob -> Alice
{'thread_id': '757e2cc1-644f-4964-b632-c3fb2bbd5689'}


#### 14.1 Check if connection state is active

In [6]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print("State", connection["state"])

Alice AGENT CONNECTION
State active


#### 14.2 Send a basic message over e2e DIDComm channel

In [10]:
#send a basic message
message = await agent_controller.messaging.send_message(alice_id,"hello from bob world!")
print("BASIC MESSAGE - Bob -> Alice")
print(message)

BASIC MESSAGE - Bob -> Alice
{}
